In [ ]:
# Install dependencies - não é necessário correr
import sys
!{sys.executable} -m pip install hdfs

In [1]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType, DoubleType
from pyspark.sql.functions import when, col, concat, lit, substring, avg, sum

client = InsecureClient("http://hdfs-nn:9870", user="anonymous")

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/fisico/gold/'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-921a8ea8-3be0-4af7-985f-1a16bae45640;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 3787ms :: artifacts dl 893ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-r

In [ ]:
# apagar tudo do hdfs

to_path = "/projeto/fisico/gold/utilization/"

client.delete(to_path,True)

In [2]:
# read data from the silver tables
df = spark.table("silver.stats")

In [ ]:
df.printSchema()
df.count()

In [3]:
# apagar todos os dados onde não há info de wins e losses
gd_df = df.filter("Year > 1946") \
          .filter("Year < 2018")

In [4]:
gd_df = gd_df.withColumn("Year", gd_df["Year"].cast(IntegerType()))

In [ ]:
gd_df.printSchema()
#gd_df.count()
#gd_df.toPandas()

In [5]:
# ficar apenas com colunas necessarias
gd_df = gd_df.select("Year","Player","Pos","Tm","Team","G","GS","MP","Wins","Losses","WP")

In [6]:
# write to delta table
gd_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000//projeto/fisico/gold/utilization/")

22/01/13 23:11:20 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
# check the results in the table
spark.table("gold.utilization").show()
spark.table("gold.utilization").count()